
```bash
# 1. Download the llamafile-ized model
wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# 2. Make executable 
chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# 3. Run in server mode
./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding --port 8080
```

In [ ]:
# Install llama-index
!pip install llama-index-core
# Install llamafile integrations and SimpleWebPageReader
!pip install llama-index-embeddings-llamafile llama-index-llms-llamafile llama-index-readers-web

In [ ]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")

Settings.llm = Llamafile(
    base_url="http://localhost:8080",
    temperature=0,
    seed=0
)

Settings.transformations = [
    SentenceSplitter(
        chunk_size=256, 
        chunk_overlap=5
    )
]

In [ ]:
# Load local data
from llama_index.core import SimpleDirectoryReader
local_doc_reader = SimpleDirectoryReader(input_dir='./data')
docs = local_doc_reader.load_data(show_progress=True)

In [ ]:
# Add Wikipedia page
from llama_index.readers.web import SimpleWebPageReader
urls = [
    'https://pl.wikipedia.org/wiki/Nad_Niemnem',
]
web_reader = SimpleWebPageReader(html_to_text=True)
docs.extend(web_reader.load_data(urls))

In [ ]:

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

index.storage_context.persist(persist_dir="./storage")